In [1]:
# -*- coding: utf-8 -*-
"""
Created on Fri Nov 29 21:02:34 2019
Pattern Recognition Action Recognition Project DT
@author: Li Meiziniu
"""    
%matplotlib inline
# import csv
import sklearn.externals
import numpy as np
from sklearn import svm
import matplotlib.pyplot as plt
from scipy.cluster.vq import *
import pandas as pd
import os
import random

In [550]:
OUTPUT_PATH = "../Output"
NUM_P_CLASS = 10
NUM_CLASS = 6
k = 4000
MAX_LENGTH = 2000

In [551]:
def readCSV(file_path):
    return np.array(pd.read_table(file_path, header=None))[:,10:-1]

def readhist(videolist,voc):
###############################
#Input: video feature, with several interests feature sequence, vocabulary obtained from kmeans
#Output: histogram of this video
#################################
    #histogram start
    im_features = np.zeros((k), "float32")
    #vq(obs, code_book[, check_finite]) Assign codes from a code book to observations.
    words, distance = vq(videolist, voc) #vector quantization
    for w in words:
        im_features[w] += 1
    return im_features

In [552]:
#Split Training set and Testing set
def createTrainTest(path):
    subdir = os.listdir(path)
    trainlist = np.array([])
    trainlabel = np.array([])
    testlist = np.array([])
    testlabel = np.array([])
    for i in subdir:
        preroot = os.path.join(path,i)
        videolist = np.array(os.listdir(preroot))
        trainIndex = random.sample(range(0, len(videolist)),NUM_P_CLASS)
        trainlist = np.hstack((trainlist, videolist[trainIndex]))
        trainlabel = np.hstack((trainlabel, [i]*NUM_P_CLASS))
        testlist = np.hstack((testlist, np.delete(videolist, trainIndex)))
        testlabel = np.hstack((testlabel, [i]*(len(videolist) - NUM_P_CLASS)))
    return trainlist, trainlabel, testlist, testlabel

trainlist, trainlabel, testlist, testlabel = createTrainTest(OUTPUT_PATH)

In [553]:
trainFeature = []
for i, j in zip(trainlist, trainlabel):
    targetVideo = os.path.join(OUTPUT_PATH, j, i)
    feature = np.vstack(readCSV(targetVideo))
    stepIndex = np.linspace(0, len(feature)-1, MAX_LENGTH, dtype = int)
    trainFeature.append(feature[stepIndex])

trainFeatureNP = np.vstack(np.array(trainFeature))

In [554]:
voc, _ = kmeans(trainFeatureNP, k, 1)

In [555]:
#Calculate the hist, use the readhist(videolist, voc) function
trainHist = []
for i in trainFeature:
    trainHist.append(readhist(i, voc))
trainHist = np.array(trainHist)

In [556]:
clf = svm.SVC()
clf.fit(trainHist,trainlabel)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [557]:
#First, lets' test train dataset
correct = 0
wrong = 0
for i in range(len(trainHist)):
    result = clf.predict(trainHist[i].reshape(1,-1))
    if result == trainlabel[i]:
        correct += 1
    else:
        wrong += 1
print("correct number is:", correct)
print("wrong number is:", wrong)
print("The Final Accuracy is:", correct/(correct+wrong))

correct number is: 60
wrong number is: 0
The Final Accuracy is: 1.0


In [558]:
#Second, let's calculate tests' feature and hist
testFeature = []
for i, j in zip(testlist, testlabel):
    targetVideo = os.path.join(OUTPUT_PATH, j, i)
    feature = np.vstack(readCSV(targetVideo))
    stepIndex = np.linspace(0, len(feature)-1, MAX_LENGTH, dtype = int)
    testFeature.append(feature[stepIndex])

#Calculate the hist, use the readhist(videolist, voc) function
testHist = []
for i in testFeature:
    testHist.append(readhist(i, voc))
testHist = np.array(testHist)

In [559]:
#lets' test train dataset
correct = 0
wrong = 0
for i in range(len(testHist)):
    result = clf.predict(testHist[i].reshape(1,-1))
    if result == testlabel[i]:
        correct += 1
    else:
        wrong += 1
print("correct number:", correct)
print("wrong number:", wrong)
print("The Final Accuracy is:", correct/(correct+wrong))

correct number: 46
wrong number: 51
The Final Accuracy is: 0.4742268041237113
